<a href="https://colab.research.google.com/github/KauaHenSilva/python_neural_networks_for_image_classification/blob/main/python_neural_networks_for_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [2]:
import requests
import os
import zipfile
import shutil

def download_large_file_from_google_drive(file_id, destination):
    base_url = "https://drive.google.com/uc?export=download"

    with requests.Session() as session:
        response = session.get(base_url, params={'id': file_id}, stream=True)
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                confirm_token = value
                break
        else:
            confirm_token = None

        if confirm_token:
            params = {'id': file_id, 'confirm': confirm_token}
            response = session.get(base_url, params=params, stream=True)

        # Escrever o conteúdo do arquivo no destino
        with open(destination, 'wb') as f:
            for chunk in response.iter_content(32768):
                f.write(chunk)

download_large_file_from_google_drive('1j5ZM-csz4Ue6xBJPcztfJ-z7Fc6eu82n', 'homer_bart.zip')

In [3]:
if not os.path.exists('homer_bart'):
    with zipfile.ZipFile('homer_bart.zip', 'r') as zip_ref:
        zip_ref.extractall('homer_bart_temp')

    os.remove('homer_bart.zip')
    shutil.move('homer_bart_temp/homer_bart_1', 'homer_bart')
    shutil.rmtree('homer_bart_temp')

Bibliotecas e pré-processamento

In [4]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from google.colab.patches import cv2_imshow
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
diretorio_imagens = './homer_bart/'
arquivos = [os.path.join(diretorio_imagens, x) for x in sorted(os.listdir(diretorio_imagens))]

In [6]:
largura, altura = 128, 128

In [7]:
images = []
classes = []

for image_caminho in arquivos:
    try:
        imagem = cv2.imread(image_caminho)
        (H, W) = imagem.shape[:2]
    except:
      continue

    imagem = cv2.resize(imagem, (largura, altura))
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    imagem = imagem.ravel()
    images.append(imagem)

    classe = os.path.basename(image_caminho)
    classe = 0 if classe.startswith('bart') else 1
    classes.append(classe)

In [8]:
X = np.array(images)
y = np.array(classes)

In [9]:
print(X.shape)
print(y.shape)

(269, 16384)
(269,)


In [10]:
np.unique(y, return_counts=True)

(array([0, 1]), array([160, 109]))

Normalizacao

In [11]:
X[0].max(), X[1].min()

(255, 0)

In [12]:
from sklearn.preprocessing import MinMaxScaler

X = MinMaxScaler().fit_transform(X)
X[0].max(), X[1].min()

(1.0000000000000002, 0.0)

Base treinamento, base Teste.


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=333)

In [14]:
X_train.shape, y_train.shape

((215, 16384), (215,))

In [15]:
X_test.shape, y_test.shape

((54, 16384), (54,))